## Part 3: Using MLP and CNNs

Add your implementation for the third part. Feel free to add your desirable functions, but please make sure you have proper functions for the final detection, where their input and output follows the same format as the previous parts.

In [9]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd7 import build_model
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
from data_generator.data_augmentation_chain_variable_input_size import DataAugmentationVariableInputSize
from data_generator.data_augmentation_chain_constant_input_size import DataAugmentationConstantInputSize
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

In [10]:
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

# The directories that contain the images.
varroa_train_images_dir  = '../datasets/project-data/images/train/'
varroa_test_images_dir   = '../datasets/project-data/images/test/'
varroa_val_images_dir    = '../datasets/project-data/images/validation/'


# The directories that contain the annotations.
varroa_train_annotations_dir = '../datasets/project-data/annotations/train/'
varroa_test_annotations_dir  = '../datasets/project-data/annotations/test/'
varroa_val_annotations_dir   = '../datasets/project-data/annotations/validation/'


# The paths to the image sets.
varroa_train_image_set_filename = '../datasets/project-data/train.txt'
varroa_test_image_set_filename  = '../datasets/project-data/test.txt'
varroa_val_image_set_filename   = '../datasets/project-data/validation.txt'



# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['background',
           'Varroa']


train_dataset.parse_xml(images_dirs=[varroa_train_images_dir],
                        image_set_filenames=[varroa_train_image_set_filename],
                        annotations_dirs=[varroa_train_annotations_dir],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False,
                        verbose=True)


val_dataset.parse_xml(images_dirs=[varroa_val_images_dir],
                        image_set_filenames=[varroa_val_image_set_filename],
                        annotations_dirs=[varroa_val_annotations_dir],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False,
                        verbose=True)

Processing image set 'validation.txt': 100%|████████████████████████████████████████| 150/150 [00:00<00:00, 301.39it/s]


In [16]:
img_height = 300 # Height of the input images
img_width = 300  # Width of the input images
img_channels = 3 # Number of color channels of the input images
intensity_mean = 127.5 # Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
intensity_range = 127.5 # Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
mean_color = [208, 202, 220] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
n_classes = 1 # Number of positive classes
scales = [0.08, 0.10, 0.12, 0.16, 0.32] # An explicit list of anchor box scaling factors. If this is passed, it will override `min_scale` and `max_scale`.
aspect_ratios = [0.5, 1.0, 2.0] # The list of aspect ratios for the anchor boxes
two_boxes_for_ar1 = True # Whether or not you want to generate two anchor boxes for aspect ratio 1
steps = None # In case you'd like to set the step sizes for the anchor box grids manually; not recommended
offsets = None # In case you'd like to set the offsets for the anchor box grids manually; not recommended
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [1.0, 1.0, 1.0, 1.0] # The list of variances by which the encoded target coordinates are scaled
normalize_coords = True # Whether or not the model is supposed to use coordinates relative to the image size

In [17]:
# 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = build_model(image_size=(img_height, img_width, img_channels),
                    n_classes=n_classes,
                    mode='training',
                    l2_regularization=0.0005,
                    scales=scales,
                    aspect_ratios_global=aspect_ratios,
                    aspect_ratios_per_layer=None,
                    two_boxes_for_ar1=two_boxes_for_ar1,
                    steps=steps,
                    offsets=offsets,
                    clip_boxes=clip_boxes,
                    variances=variances,
                    normalize_coords=normalize_coords,
                    subtract_mean=intensity_mean,
                    divide_by_stddev=intensity_range)

# 2: Optional: Load some weights

#model.load_weights('./ssd7_weights.h5', by_name=True)

# 3: Instantiate an Adam optimizer and the SSD loss function and compile the model

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [18]:
# 3: Set the batch size.

batch_size = 10

# 4: Define the image processing chain.

# For the training generator:
# data_augmentation_chain = DataAugmentationVariableInputSize(resize_height=img_height,
#                                            resize_width=img_width)

data_augmentation_chain = SSDDataAugmentation(img_height=img_height,img_width=img_width,
                                            background=mean_color)

# For the validation generator:
resize = Resize(height=img_height, width=img_width)


# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('classes4').output_shape[1:3],
                   model.get_layer('classes5').output_shape[1:3],
                   model.get_layer('classes6').output_shape[1:3],
                   model.get_layer('classes7').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_global=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.3,
                                    normalize_coords=normalize_coords)


# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[
                                                      data_augmentation_chain],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	   800
Number of images in the validation dataset:	   150


In [19]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the weights.
model_checkpoint = ModelCheckpoint(filepath='ssd7_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)

csv_logger = CSVLogger(filename='ssd7_training_log.csv',
                       separator=',',
                       append=True)

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=10,
                               verbose=1)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.2,
                                         patience=8,
                                         verbose=1,
                                         epsilon=0.001,
                                         cooldown=0,
                                         min_lr=0.00001)

callbacks = [model_checkpoint,
             csv_logger,
             early_stopping,
             reduce_learning_rate]

C:\Users\Ismail-P51\Anaconda3\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [ ]:
# TODO: Set the epochs to train for.
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 10
steps_per_epoch = 300

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Epoch 1/10
220/300 [=====================>........] - ETA: 34:45 - loss: 22.356 - ETA: 20:59 - loss: 20.707 - ETA: 17:17 - loss: 19.036 - ETA: 19:34 - loss: 17.651 - ETA: 17:24 - loss: 16.723 - ETA: 16:43 - loss: 16.030 - ETA: 16:11 - loss: 15.533 - ETA: 16:39 - loss: 14.884 - ETA: 15:54 - loss: 14.523 - ETA: 15:37 - loss: 14.372 - ETA: 16:34 - loss: 13.986 - ETA: 15:45 - loss: 13.644 - ETA: 15:48 - loss: 13.288 - ETA: 15:14 - loss: 12.917 - ETA: 15:12 - loss: 12.550 - ETA: 15:02 - loss: 12.389 - ETA: 14:53 - loss: 12.079 - ETA: 14:45 - loss: 11.820 - ETA: 14:53 - loss: 11.641 - ETA: 14:39 - loss: 11.419 - ETA: 14:46 - loss: 11.217 - ETA: 14:30 - loss: 11.016 - ETA: 14:21 - loss: 10.803 - ETA: 14:15 - loss: 10.608 - ETA: 14:06 - loss: 10.424 - ETA: 14:01 - loss: 10.282 - ETA: 13:50 - loss: 10.153 - ETA: 13:42 - loss: 9.987 - ETA: 13:37 - loss: 9.85 - ETA: 13:29 - loss: 9.70 - ETA: 13:39 - loss: 9.60 - ETA: 13:31 - loss: 9.51 - ETA: 13:36 - loss: 9.42 - ETA: 13:26 - loss: 9.34 - ETA: 13

In [ ]:
plt.figure(figsize=(20,12))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(loc='upper right', prop={'size': 24});

In [ ]:
# 1: Set the generator for the predictions.

predict_generator = val_dataset.generate(batch_size=1,
                                         shuffle=True,
                                         transformations=[convert_to_3_channels,
                                                          resize],
                                         label_encoder=None,
                                         returns={'processed_images',
                                                  'filenames',
                                                  'inverse_transform',
                                                  'original_images',
                                                  'original_labels'},
                                         keep_images_without_gt=False)

In [ ]:
# 2: Generate samples.

batch_images, batch_filenames, batch_inverse_transforms, batch_original_images, batch_original_labels = next(predict_generator)

i = 0 # Which batch item to look at

print("Image:", batch_filenames[i])
print()
print("Ground truth boxes:\n")
print(np.array(batch_original_labels[i]))

In [ ]:
# 3: Make predictions.

y_pred = model.predict(batch_images)

In [ ]:
# 4: Decode the raw prediction `y_pred`

y_pred_decoded = decode_detections(y_pred,
                                   confidence_thresh=0.3,
                                   iou_threshold=0.1,
                                   top_k=200,
                                   normalize_coords=normalize_coords,
                                   img_height=img_height,
                                   img_width=img_width)

np.set_printoptions(precision=2, suppress=True, linewidth=90)
print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')
print(y_pred_decoded[i])

In [ ]:
# 5: Convert the predictions for the original image.

y_pred_decoded_inv = apply_inverse_transforms(y_pred_decoded, batch_inverse_transforms)

np.set_printoptions(precision=2, suppress=True, linewidth=90)
print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')
print(y_pred_decoded_inv[i])

In [ ]:
# 5: Draw the predicted boxes onto the image

# Set the colors for the bounding boxes
colors = plt.cm.hsv(np.linspace(0, 1, n_classes+1)).tolist()
classes = ['background',
           'Varroa']
plt.figure(figsize=(20,12))
plt.imshow(batch_original_images[i])

current_axis = plt.gca()

for box in batch_original_labels[i]:
    xmin = box[1]
    ymin = box[2]
    xmax = box[3]
    ymax = box[4]
    label = '{}'.format(classes[int(box[0])])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='green', fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':'green', 'alpha':1.0})

for box in y_pred_decoded_inv[i]:
    xmin = box[2]
    ymin = box[3]
    xmax = box[4]
    ymax = box[5]
    color = colors[int(box[0])]
    label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})

Need to know the fucking parameters on the images

In [ ]:
print(val_ic[5].shape)
print(val_xmls[5])
fig, ax = plt.subplots()
plt.imshow(val_ic[5])  
rect = patches.Rectangle((val_xmls[5][0]['bbox'][0], val_xmls[5][0]['bbox'][1]), val_xmls[5][0]['bbox'][2], val_xmls[5][0]['bbox'][3],
                linewidth=1,edgecolor='r',facecolor='none')
ax.add_patch(rect)

In [ ]:
# Reshape all the images to 1024x1024 and conserv aspect ratio
from skimage.transform import resize
def reshape_img_bb(img,bbox_list):
    DIM_IMG = 1024
    
    ratio_x = DIM_IMG/img.shape[1]
    ratio_y = DIM_IMG/img.shape[0]
    result_img = resize(img,(DIM_IMG,DIM_IMG),anti_aliasing=True)
    result_boxes = []
    if 0 != len(bbox_list):
        for elem in bbox_list:
            new_x = int(elem['bbox'][0] * ratio_x)
            new_y = int(elem['bbox'][1] * ratio_y)
            new_w = int(elem['bbox'][2] * ratio_x)
            new_h = int(elem['bbox'][3] * ratio_x)
            result_boxes.append((new_x,new_y,new_w,new_h))
            
    return (result_img,result_boxes)

Fucking validate the method

In [ ]:
ax = plt.subplot(121)
for elem in tst_xmls[2]:
    rect = patches.Rectangle((elem['bbox'][0], elem['bbox'][1]), elem['bbox'][2], elem['bbox'][3],
                    linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
ax.imshow(tst_ic[2])

ax = plt.subplot(122)
print(tst_ic[2].shape)
print(tst_xmls[2])
n_img,n_boxes = reshape_img_bb(tst_ic[2],tst_xmls[2])

for elem in n_boxes:
    rect = patches.Rectangle((elem[0], elem[1]), elem[2], elem[3],
                    linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
ax.imshow(n_img)

Add SDD7 support and make it fucking work

In [ ]:
model = Sequential([
        Dense(200, input_dim=64), 
        Activation('relu'), 
        Dropout(0.2), 
        Dense(4)
    ])
model.compile('adadelta', 'mse')

## Challenge

You can generate a json submission file by using the function ''**generate_pred_json**''. This prediction file can be uploaded online for evaluation (Please refer to section 3 of the project description for more details).

In [ ]:
import numpy as np
import json

def generate_pred_json(data, tag='baseline'):
    '''
    Input
    - data: Is a dictionary d, such that:
          d = { 
              "ID_1": [], 
              "ID_2": [[x_21, y_21, w_21, h_21], [x_22, y_22, w_22, h_22]], 
              ... 
              "ID_i": [[x_i1, y_i1, w_i1, h_i1], ..., [x_iJ, y_iJ, w_iJ, h_iJ]],
              ... 
              "ID_N": [[x_N1, y_N1, w_N1, h_N1]],
          }
          where ID is the string id of the image (e.i. 5a05e86fa07d56baef59b1cb_32.00px_1) and the value the Kx4 
          array of intergers for the K predicted bounding boxes (e.g. [[170, 120, 15, 15]])
    - tag: (optional) string that will be added to the name of the json file.
    Output
      Create a json file, "prediction_[tag].json", conatining the prediction to EvalAI format.
    '''
    unvalid_key = []
    _data = data.copy()
    for key, value in _data.items():
        try:
            # Try to convert to numpy array and cast as closest int
            print(key)
            v = np.around(np.array(value)).astype(int)
            # Check is it is a 2d array with 4 columns (x,y,w,h)
            if v.ndim != 2 or v.shape[1] != 4:
                unvalid_key.append(key)
            # Id must be a string
            if not isinstance(key, str):
                unvalid_key.append(key)
            _data[key] = v.tolist()
        # Deal with not consistant array size and empty predictions
        except (ValueError, TypeError):
            unvalid_key.append(key)
    # Remove unvalid key from dictionnary
    for key in unvalid_key: del _data[key]
    
    with open('prediction_{}.json'.format(tag), 'w') as outfile:
        json.dump(_data, outfile)

In [ ]:
#Your code